### Object tracking with opencv

In [1]:
## first step was finding objects, now its time for tracking

In [44]:
import cv2

In [45]:
def initialize(videopath):
    cap = cv2.VideoCapture(videopath)
    od = cv2.createBackgroundSubtractorMOG2(varThreshold=50, history=100) # better selection and quicker
    # od = cv2.createBackgroundSubtractorKNN()
    return cap, od

In [46]:
def cleanup_frame(frame, od):
    # frame = cv2.resize(frame, (960, 540))                             # Resize image
    mask = od.apply(frame)                                              # run mask (black and white)
    _, extra_mask = cv2.threshold(mask, 250, 255, cv2.THRESH_BINARY)    # set treshold to only get really contrasting values in mask
    return extra_mask

In [47]:
def identifying_objects(c, od):
    count = 0
    frame_nr = 6
    while True:
        count +=1
        print(c.get(1))     
        ret, frame = c.read()                                       # read frames
        if not ret:
            print("Stream ended. Exiting...")
            break        
        key = cv2.waitKey(1)
        if key == ord('q'):                                         # exit
            break        
        elif key == ord('x'):                                       # pauze screen
            cv2.waitKey(0)        
        # frame = cv2.resize(frame, (960, 540))                       # Resize image
        # cv2.imshow('frame', frame)
        mask = od.apply(frame)                                      # run mask (black and white)
        _, mask = cv2.threshold(mask, 254, 255,cv2.THRESH_BINARY)   # set treshold to only get really contrasting values in mask
        cv2.imshow('mask', mask)
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #Each individual contour is a Numpy array of (x,y) coordinates of boundary points of the object.
        frame_detect = []
        if count == frame_nr:
            for cnt in contours:
                print('getting contours')
                x,y,w,h = cv2.boundingRect(cnt)                         # get boundingbox
                frame_detect.append([x,y,w,h])
                cv2.rectangle(frame,(x,y), (x+w, y+h),(0,0, 255), 3)     # draw rectangle
                cv2.imshow('frame2', frame)    
            break
    c.release()
    cv2.destroyAllWindows()
    [cv2.waitKey(1) for i in range(1,5)]                            # needed to destroy window
    return frame_detect

In [48]:
cap, od =initialize('../data/aerial_football_video.mp4')

frame_bbox = identifying_objects(cap, od)

0.0
1.0
2.0
3.0
4.0
5.0
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting contours
getting

In [49]:
len(frame_bbox)

90

In [50]:
frame_bbox[:5]

[[837, 638, 3, 2],
 [837, 636, 1, 1],
 [835, 635, 1, 3],
 [837, 633, 1, 1],
 [695, 543, 2, 2]]

In [51]:
frame_bbox_large = []
for bb in frame_bbox:
    widthlarge = bb[2] * 50
    heightlarge = bb[3] * 50
    frame_bbox_large.append((bb[0], bb[1], widthlarge, heightlarge))

In [52]:
frame_bbox_large[:5]

[(837, 638, 150, 100),
 (837, 636, 50, 50),
 (835, 635, 50, 150),
 (837, 633, 50, 50),
 (695, 543, 100, 100)]

In [53]:
cap, od =initialize('../data/aerial_football_video.mp4')
ret, frame_n = cap.read()  # Read the frame

for start_contour in frame_bbox_large:
    bbox = start_contour
    # bbox = (546, 109, 150, 150)
    tracker=cv2.legacy.TrackerMOSSE_create()
    ok = tracker.init(frame_n ,bbox)
    first = None
    while True:
        if not first:
            cap.set(1,n+1) #otherwise starts  reading from 22 when starting with 20
            first ='done'
        ret, frame = cap.read() 
        # frame = cleanup_frame(frame, od)
        # print(cap.get(1))
        if not ret:
            break
        ok, bbox = tracker.update(frame)
        if ok:
            print('tracking!')
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (0,0,255), 2, 1)
        else :
            print('not tracked')
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
        cv2.putText(frame, "Mosse" + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2)
        cv2.imshow("Tracking", frame)
        key = cv2.waitKey(1)
        if key == ord('q'):                                         # exit
            break        
        elif key == ord('x'):                                       # pauze screen
            cv2.waitKey(0)            

    # cap.release()
    # cv2.destroyAllWindows()
    # [cv2.waitKey(1) for i in range(1,5)]           


not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not tracked
not 

In [37]:
# cap, od =initialize('../data/aerial_football_video.mp4')
# n=20
# framenr = cap.get(1)
# print(framenr)
# cap.set(1,n)  # Where frame_no is the frame you want
# ret, frame_n = cap.read()  # Read the frame
# # succes, first_frame = cap.read()
# # frame100 = cleanup_frame(frame100, od)
# # bbox=cv2.selectROI(frame_n)
# # bbox = (424, 278, 54, 32)  # test case
# # bbox = (837, 638, 3, 2)
# # bbox = (545, 112, 50, 50)
# bbox = (546, 109, 150, 150)
# print(bbox)
# # bbox = first_contours(first_frame)
# tracker=cv2.legacy.TrackerMOSSE_create()
# ok = tracker.init(frame_n ,bbox)
# first = None
# while True:
#     if not first:
#         cap.set(1,n+1) #otherwise starts  reading from 22 when starting with 20
#         first ='done'
#     ret, frame = cap.read() 
#     # frame = cleanup_frame(frame, od)
#     print(cap.get(1))
#     if not ret:
#         break
#     ok, bbox = tracker.update(frame)
#     print(ok)
#     if ok:
#         print('tracking!')
#         p1 = (int(bbox[0]), int(bbox[1]))
#         p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
#         cv2.rectangle(frame, p1, p2, (0,0,255), 2, 1)
#     else :
#         print('not tracked')
#         cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
#     cv2.putText(frame, "Mosse" + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2)
#     cv2.imshow("Tracking", frame)
#     key = cv2.waitKey(1)
#     if key == ord('q'):                                         # exit
#         break        
#     elif key == ord('x'):                                       # pauze screen
#         cv2.waitKey(0)            

# cap.release()
# cv2.destroyAllWindows()
# [cv2.waitKey(1) for i in range(1,5)]           


0.0
(546, 109, 150, 150)
22.0
False
23.0
False
24.0
False
25.0
False
26.0
False
27.0
False
28.0
False
29.0
False
30.0
False
31.0
False
32.0
False
33.0
False
34.0
False
35.0
False
36.0
False
37.0
False
38.0
False
39.0
False
40.0
False
41.0
False
42.0
False
43.0
False
44.0
False
45.0
False
46.0
False
47.0
False
48.0
False
49.0
False
50.0
False
51.0
False
52.0
False
53.0
False
54.0
False
55.0
False
56.0
False
57.0
False
58.0
False
59.0
False
60.0
False
61.0
False
62.0
False
63.0
False
64.0
False
65.0
False
66.0
False
67.0
False
68.0
False
69.0
False
70.0
False
71.0
False
72.0
False
73.0
False
74.0
False
75.0
False
76.0
False
77.0
False
78.0
False
79.0
False
80.0
False
81.0
False
82.0
False
83.0
False
84.0
False
85.0
False
86.0
False
87.0
False
88.0
False
89.0
False
90.0
False
91.0
False
92.0
False
93.0
False
94.0
False
95.0
False
96.0
False
97.0
False
98.0
False
99.0
False
100.0
False
101.0
False
102.0
False
103.0
False
104.0
False
105.0
False
106.0
False
107.0
False
108.0
False
109.0
Fal

[113, -1, -1, -1]

In [6]:
#(424, 278, 54, 32) frame 20 starting point tracking bbox